On HSLU GPUHUB You must use the server template __CVAI admin__

## Declare var + functions (run each time)

In [ ]:
import os
import git
import shutil
from pathlib import Path

# Home path (only works on Linux)
# https://stackoverflow.com/a/58988310
home_dir = Path(os.readlink('/proc/%s/cwd' % os.environ['JPY_PARENT_PID']))

# Paths
sd_dir = home_dir / 'stable-diffusion-webui'
sd_model_dir = sd_dir / 'models'
comfyui_dir = home_dir / 'ComfyUI'

In [ ]:
# Clone a repo
def clone_repo(git_url, target_dir):
    
    # Extract repo name and create path
    repo_name = git_url.split('/')[-1]

    # Create path
    repo_dir = target_dir / repo_name

    # Clone if does not exist
    if not repo_dir.exists():
        git.Repo.clone_from(git_url, repo_dir)
    else:
        print('Directory exists already, skipping')


# Delete a whole dir and it'd content
def del_dir(dir):
    answer = input(f"-> Are you sure you want to delete '{dir}' [y | n]: ")
    if answer == 'y':
        if dir.exists():
            shutil.rmtree(dir)
            print('Directory deleted')
        else:
            print('Directory not found')
    else:
        print('Skipping')

## Clone repo and setup Comfyui (run once)

Clone Repo

In [ ]:
clone_repo('https://github.com/comfyanonymous/ComfyUI', home_dir)

### If SD-webui is installed we can use the models from there

Show the stable diffusion webui dir:

In [ ]:
sd_dir

Manually modify the file `ComfyUI/extra_model_paths.yaml.example`

1) Change `base_path: path/to/stable-diffusion-webui/` with the path from our install (see the value of the var `sd_dir` ⬆️)
2) Rename file to `ComfyUI/extra_model_paths.yaml`

## Install packages (run each time)

In [ ]:
!nvidia-smi

Upgrade pip (not a bad idea :P )

In [ ]:
!pip install --upgrade --quiet pip

Install xformers to speed up inference and huggingface-hub

In [ ]:
!pip install --quiet xformers==0.0.22 huggingface-hub

Install requirements

In [ ]:
!pip install --quiet -r {comfyui_dir}/requirements.txt

Install torchvision with GPU support

In [ ]:
!pip install --quiet torch==2.0.1+cu117 torchvision==0.15.2+cu117 --index-url https://download.pytorch.org/whl/cu117

Check if GPU is working

In [ ]:
import torch
torch.cuda.is_available()

If this does not show `True` you might need to restart the kernel `torch.__version__` should show `2.0.1+cu117`

In [ ]:
torch.__version__

## Install extensions and download models (run once)

### Extensions

Manager for custom nodes and then some...

In [ ]:
clone_repo('https://github.com/ltdrdata/ComfyUI-Manager', comfyui_dir / 'custom_nodes')

ComfyUI-Custom-Scripts

In [ ]:
clone_repo('https://github.com/pythongosssss/ComfyUI-Custom-Scripts', comfyui_dir / 'custom_nodes')

Animate diff

In [ ]:
clone_repo('https://github.com/Kosinkadink/ComfyUI-AnimateDiff-Evolved', comfyui_dir / 'custom_nodes')

### Models

Download models for AnimateDiff  
https://github.com/Kosinkadink/ComfyUI-AnimateDiff-Evolved#how-to-use

In [ ]:
import huggingface_hub

repo_id='guoyww/animatediff'

# Motion modules
local_dir = comfyui_dir / 'custom_nodes/ComfyUI-AnimateDiff-Evolved/models'

huggingface_hub.snapshot_download(
    repo_id=repo_id,
    local_dir=local_dir,
    allow_patterns="mm_sd*.ckpt"
)

# Motion Lora
local_dir = comfyui_dir / 'custom_nodes/ComfyUI-AnimateDiff-Evolved/motion_lora'

huggingface_hub.snapshot_download(
    repo_id=repo_id,
    local_dir=local_dir,
    allow_patterns="v2_lora*.ckpt"
)

We will need to install some nodes manually afterwards, see:  
https://github.com/Kosinkadink/ComfyUI-AnimateDiff-Evolved#animatediff-for-comfyui

Autoencoder used for SD

In [ ]:
# SD models
local_dir = sd_model_dir / 'VAE'

# SD VAE
repo_id='stabilityai/sd-vae-ft-mse-original'
filename = 'vae-ft-mse-840000-ema-pruned.safetensors'
huggingface_hub.hf_hub_download(
    repo_id=repo_id,
    filename=filename,
    local_dir=local_dir
)

A nice / fun SD model, we download it in the default stable-diffusion model dir 

In [ ]:
# https://civitai.com/models/7371/rev-animated
repo_id='s6yx/ReV_Animated'
local_dir = sd_model_dir / 'Stable-diffusion'
filename = 'rev_1.2.2.safetensors'
subfolder = 'rev_1.2.2'

huggingface_hub.hf_hub_download(
    repo_id=repo_id,
    filename=filename,
    subfolder=subfolder,
    local_dir=local_dir
)

We move it to the root of the sd models dir, don't really have to but it's "cleaner" this way

In [ ]:
!mv {local_dir}/{subfolder}/{filename} {local_dir}

## 📍 Run

You first need to start your `Ngrok` notebook on the side to be able to access the UI, default port is `8188`

In [ ]:
os.chdir(comfyui_dir)

In [ ]:
!python main.py